In [119]:
import os
import pandas as pd
from collections import Counter
import numpy as np
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem.snowball import EnglishStemmer
import re
import itertools

In [120]:
# function to perform tokenization and Normalization
def token_norm(content):
    content_token = []     # converting each review as a list of tokens and creating a list of, list of tokens
    for i in range(0, len(content)):
#        s = re.sub('([.])', r' \1 ', content[i])
#        tokenizer = nltk.tokenize.treebank.TreebankWordTokenizer()
        tokenizer = RegexpTokenizer(r'\w+')  # to remove Punctuations
        c = tokenizer.tokenize(content[i])
        content_token.append(c)
        
    #Normalization, Stemming and Stopword Removal
    content_norm = []
    for i in range(0,len(content_token)):
        con = []
        for item in content_token[i]:
            l=item.lower()                # converting all items to lower case
            l1=re.sub("[\d]+","NUM", l)   # converting all digits to 'NUM'
            stemmer = EnglishStemmer()    # get an instance of SnowballStemmer for English
            l2=stemmer.stem(l1)           # Stemming
#            if l2 not in stopwords_set:   # Stopword list 
            con.append(l2)
        content_norm.append(con)
    return(content_norm)

In [121]:
def token_norm_string(string):
#    s = re.sub('([.])', r' \1 ', content[i])
#    tokenizer = nltk.tokenize.treebank.TreebankWordTokenizer()
    tokenizer = RegexpTokenizer(r'\w+')  # to remove Punctuations
    c = tokenizer.tokenize(string)
        
# Normalization, Stemming and Stopword Removal
    con = []
    for item in c:
        l=item.lower()                # converting all items to lower case
        l1=re.sub("[\d]+","NUM", l)   # converting all digits to 'NUM'
        stemmer = EnglishStemmer()    # get an instance of SnowballStemmer for English
        l2=stemmer.stem(l1)           # Stemming
#       if l2 not in stopwords_set:   # Stopword list 
        con.append(l2)
    return(con)

In [122]:
path = '/home/prateek/uva/Data Visualization/Speeches/All Speeches'

In [123]:
speeches = [filelist for filelist in os.listdir(path)]

In [125]:
# creating a dictionary for each speech
df = {}
for sp in speeches:
    with open(os.path.join(path, sp)) as speech:
        content = speech.read()
        president = sp.split('-',1)[0]
        if president in df.keys():
            df[president][sp[:-4]] = content
            
        else:
            df[president] = {}
            df[president][sp[:-4]] = content

In [132]:
speech_df = pd.DataFrame([[col1, col2, col3] for col1, d in df.items() for col2, col3 in d.items()])
speech_df.columns = ['President','Speech Title','Speech']
speech_df['Year'] = [year[-5:-1] for year in speech_df['Speech Title']]
speech_df.head()

,President,Speech Title,Speech,Year
0,reagan,"reagan-""Evil Empire"" Speech (March 8, 1983)","Reverend clergy all, Senator Hawkins, disting...",1983
1,reagan,reagan-Bergen-Belsen Concentration Camp (May 5...,"Chancellor Kohl and honored guests, this pain...",1985
2,reagan,"reagan-Farewell Address (January 11, 1989)",This is the 34th time I'll speak to you from ...,1989
3,reagan,reagan-Speech on the Strategic Arms Reduction ...,"Officers, ladies and gentlemen of the Nationa...",1981
4,reagan,reagan-Address from the Brandenburg Gate (Berl...,"Thank you very much. Chancellor Kohl, Governi...",1987


In [133]:
speech_df['unique words'] = [len(Counter(token_norm_string(d))) for d in speech_df['Speech']]
speech_df['total words'] = [sum(Counter(token_norm_string(d)).values()) for d in speech_df['Speech']]

In [134]:
speech_df.head()

,President,Speech Title,Speech,Year,unique words,total words
0,reagan,"reagan-""Evil Empire"" Speech (March 8, 1983)","Reverend clergy all, Senator Hawkins, disting...",1983,1064,3932
1,reagan,reagan-Bergen-Belsen Concentration Camp (May 5...,"Chancellor Kohl and honored guests, this pain...",1985,426,1165
2,reagan,"reagan-Farewell Address (January 11, 1989)",This is the 34th time I'll speak to you from ...,1989,849,3401
3,reagan,reagan-Speech on the Strategic Arms Reduction ...,"Officers, ladies and gentlemen of the Nationa...",1981,840,3293
4,reagan,reagan-Address from the Brandenburg Gate (Berl...,"Thank you very much. Chancellor Kohl, Governi...",1987,797,2721


In [135]:
president_list = list(speech_df['President'].unique())

In [136]:
president_list

['reagan',
 'gwbush',
 'clinton',
 'bush',
 'obama',
 'nixon',
 'ford',
 'lbjohnson',
 'carter']

In [99]:
#d1 = Counter(token_norm_string(list(speech_df['Speech'])[1]))

In [141]:
Pred_data = pd.DataFrame(columns=['President Name','Year','Total Unique Words','Total Words'])
for name in president_list:
    token_list= []
    year = []
    for i in range(len(speech_df)):
        if speech_df.loc[i,'President'] == name:
            token_list.append(token_norm_string(speech_df.loc[i,'Speech']))
            year.append(speech_df.loc[i, 'Year'])
    max_year = max(year)
    min_year = min(year)
    total_wordlist =  list(itertools.chain.from_iterable(token_list))
    Pred_data.loc[president_list.index(name)] = [name, min_year+'-'+max_year, len(Counter(total_wordlist)), sum(Counter(total_wordlist).values()) ]

In [142]:
Pred_data.head()

,President Name,Year,Total Unique Words,Total Words
0,reagan,1964-1989,6830,219135
1,gwbush,2001-2009,3323,46159
2,clinton,1993-2001,5197,150100
3,bush,1988-1993,4439,98040
4,obama,2008-2010,2904,34632


In [98]:
#p = speech_df.groupby(['President'], axis=0)['total words'].sum()
#p

President
obama    34632
Name: total words, dtype: int64